# 2. Линейная регрессия

**Цель работы**: Изучить основные понятия машинного обучения, исследовать методы решения задачи регрессии, применить полученные знания для решения практических задач.

## Регрессия

### Обучающая выборка

Задача регрессии заключается в поиске зависимости некоторой переменной $y$ от другой переменной $x$. При этом переменная $x$ может быть векторной.

$$x=(x_1, x_2, \dots, x_n).$$

В этом случае говорят о *множественной регрессии*. В противном случае, если $x$ — скаляр, регрессию называют *парной*.

Компоненты $x_j$ называются *признаками*.

Набор данных, который используется для восстановления зависимости называется *обучающей выборкой*. Она представляет собой пару $(X, Y)$, где

$$X = 
\left(
\begin{array}%
x^{(1)}\\
x^{(2)}\\
\vdots\\
x^{(m)}\\
\end{array}
\right) = 
\left(
\begin{array}%
x^{(1)}_1&x^{(1)}_2&\dots&x^{(1)}_n\\
x^{(2)}_1&x^{(2)}_2&\dots&x^{(2)}_n\\
\vdots&\vdots&\ddots&\vdots\\\
x^{(m)}_1&x^{(m)}_2&\dots&x^{(m)}_n\\
\end{array}
\right),
Y = 
\left(
\begin{array}%
y^{(1)}\\
y^{(2)}\\
\vdots\\
y^{(m)}\\
\end{array}
\right).
$$

Пара $(x^{(i)}, y^{(i)})$ называется *прецедентом*.

### Линейная регрессия

Простейший случай регрессии — линейная регрессия. В ней искомая зависимость описывается линейной функцией.

$$h_{\theta}(x) = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n = \theta_0 + \sum_{j=1}^n\theta_jx_j.$$

Здесь $h_{\theta}(x)$ — обучаемая (в данном случае линейная) модель описывающая зависимость $y$ от $x$. Параметры $\theta_j$ — параметры модели, а $\theta$ — вектор параметров.

$$\theta = \left(
\begin{array}%
\theta_0\\
\theta_1\\
\theta_2\\
\vdots\\
\theta_n\\
\end{array}
\right).
$$

Для упрощения записи выражений можно ввести фиктивный признак 

$$x_0 \equiv 1.$$

Этот признак добавляется к исходным данным как столбец из 1 в матрице $X$.

Тогда модель записывается как скалярное произведение

$$h_{\theta}(x) = \sum_{j=1}^m x_j \theta_j = x \theta.$$

Значения, предсказанные моделью для каждого набора признаком из обучающей выборки можно вычислить аналогично.

$$h_{\theta}(X) = X \theta.$$

В этом случае получаем вектор-столбец со значениями $h_{\theta}(x^{(i)})$ для всех $i$.

### Функция потерь

Параметры $\theta$ выбираются таким образом, чтобы минимизировать ошибку между предсказанными ($h_{\theta}(X)$) и известными в обучающей выборке ($Y$) значениями.

Часто в качестве меры такой ошибки берут среднеквадратическое отклонение.

$$J(\theta) = \frac1{2m}\sum_{i=1}^{m}\left[h_{\theta}(x^{(i)})-y^{(i)}\right]^2$$

Функция $J(\theta)$ называется *функцией потерь*.

Тогда параметры модели находят как аргумент минимума функции потерь

$$\theta = \arg\min_{\theta}J(\theta).$$

## Задание

В настоящей лабораторной работе требуется решить задачу поиска оптимального ветора $\theta$ с помощью следующих методов:

- нормальное уравнение,
- метод градиентного спуска,
- метод BFGS (алгоритм реализован в библиотеке SciPy).

Дополнительное задание: попробуйте нормировать исходные данные. Не забудьте сохранить параметры, с которыми выполнялась нормализация (среднее значение и разброс).

## Подготовка библиотек

In [ ]:
import numpy as np
import scipy.io as sio
import scipy.optimize as so
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
rc('font', family='Verdana')

## Задача 1. Зависимость роста от возраста

Требуется найти зависимость роста (в метрах) от возраста (в годах) детей.

Загрузим обучающую выборку.

In [ ]:
data = sio.loadmat('heights.mat')
Xdata = data['age']
ydata = data['height']

# Xp, Yp задают линию, которая будет отображена на графике поверх данных
def plot_data(X, y, Xp=None, yp=None):
    plt.plot(X, y, 'xr')
    if Xp is not None and yp is not None:
        plt.plot(Xp, yp, '-b')
    plt.xlabel('Возраст, лет')
    plt.ylabel('Рост, м')
    plt.title('Зависимость роста от возраста')
    plt.show()

plot_data(Xdata, ydata)

Введём вспомогательные величины: $m$ — число прецедентов, $n$ — номер последнего признака. (В функциях ими пользоваться нежелательно, так как функции должны быть как можно более изолированы от остального кода.)

In [ ]:
m, n = Xdata.shape

Добавим к матрице $X$ слева столбец из единиц с помощью функции `np.concatenate`. Не забудьте в дальнейшем при расчётах использовать матрицу `X1` вместо `X`.

In [ ]:
Xdata1 = np.concatenate([np.ones((m,1)), Xdata], axis=1)

Запишите выражение для модели $h_{\theta}(x)$.

$x$ может быть как вектор-строкой размера $1\times (n+1)$, так и матрицей (то есть набором вектор-строк).

$\theta$ — вектор-столбец размера $(n+1)\times 1$.

Функция должна возвращать предсказанное значение для каждой строки параметра `X`.

In [ ]:
def h(X, theta):
    # Проверки входных данных
    assert len(X.shape) == 2, "X — матрица?"
    assert X.shape[1] == n+1, "Неверный размер матрицы X. Забыли столбец из 1?" 
    assert theta.shape == (n+1, 1), "Неверный размер матрицы theta"
    
    # ИСПРАВЬТЕ ИЛИ ДОПОЛНИТЕ КОД НИЖЕ
    return np.ones((X.shape[0], 1))
    ###

Проверим работу функции для прямой пропорциональности. Параметры $\theta$ выбраны произвольно и не соответствуют искомому решению.

In [ ]:
theta = np.array([[1], [0.1]])
yp = h(Xdata1, theta) # Предсказанные значения
plot_data(Xdata, ydata, Xdata, yp)

### Нормальное уравнение
Найдём $\theta$ с помощью нормального уравнения.

In [ ]:
# ИСПРАВЬТЕ ИЛИ ДОПОЛНИТЕ КОД НИЖЕ
theta_norm = np.array([[0], [1]])
###

Графически оценим результаты вычислений.

In [ ]:
print (theta_norm)
yp = h(Xdata1, theta_norm) # Предсказанные значения
plot_data(Xdata, ydata, Xdata, yp)

### Градиентный спуск

Найдём $\theta$ методом градиентного спуска. Для этого определим функции `J` и `dJ`, соответствующие функции потерь и вектору градиента.

Функция `J` принимает на входе как вектор-столбец параметров `theta`, так и данные обучающей выборки — `X` и `y`. Функция должна вернуть число, характеризующее среднюю ошибку предсказанного моделью значения от истинного.

In [ ]:
def J(theta, X, y):
    # Проверки входных данных
    assert len(X.shape) == 2, "X — матрица?"
    assert X.shape[1] == n+1, "Неверный размер матрицы X. Забыли столбец из 1?" 
    assert X.shape[0] == y.shape[0], "Разное количество выходных и входных переменных"
    assert theta.shape == (n+1, 1), "Неверный размер матрицы theta"
    
    # ИСПРАВЬТЕ ИЛИ ДОПОЛНИТЕ КОД НИЖЕ
    return 0.0
    ###

Параметры функции, вычисляющей градиент, те же, но она должна вернуть вектор-столбец со значениями производной $\frac{\partial J(\theta)}{\partial\theta_j}$ по каждому из параметров $\theta_j$. Этот вектор-столбец, очевидно, должен иметь те же размеры, что и вектор $\theta$.

In [ ]:
def dJ(theta, X, y):
    # Проверки входных данных
    assert len(X.shape) == 2, "X — матрица?"
    assert X.shape[1] == n+1, "Неверный размер матрицы X. Забыли столбец из 1?" 
    assert X.shape[0] == y.shape[0], "Разное количество выходных и входных переменных"
    assert theta.shape == (n+1, 1), "Неверный размер матрицы theta"
    
    # ИСПРАВЬТЕ ИЛИ ДОПОЛНИТЕ КОД НИЖЕ
    return np.zeros(theta.shape)
    ###

Сам градиентный спуск выполняется достаточно просто: выполняются шаги в сторону, противоположную направлению градиента, пока его норма не станет достаточно малой (то есть, не станет меньше некоторого $\varepsilon$). Конечно, за простоту приходится платить — более совершенные методы точнее и быстрее сходятся.

Для работы метода необходимо правильно подобрать параметры $\varepsilon$ (параметр, управляющей точностью) и $\alpha$ (коэффициент шага градиентного спуска).

In [ ]:
eps = 0.01
# НАЙДИТЕ ОПТИМАЛЬНОЕ ЗНАЧЕНИЕ КОЭФФИЦИЕНТА alpha
alpha = 0.3

Основной алгоритм выглядит следующим образом.

В массиве `norms` накапливаются значения нормы на каждом шаге цикла. Они нужны для точной «подстройки» парметра $\alpha$.

Если число итераций превышает `Kmax`, алгоритм завершается принудительно.

In [ ]:
k, Kmax = 0, 10000
norms = []

# Инициализация
theta_grad = np.zeros((n+1, 1))
while True:
    d = dJ(theta_grad, Xdata1, ydata) # Текущее значение градиента
    
    norm = np.linalg.norm(d)
    norms.append(norm)
    k += 1
    if k > Kmax or norm < eps: break

    # Один шаг градиентного спуска
    # ИСПРАВЬТЕ ИЛИ ДОПОЛНИТЕ КОД НИЖЕ
    theta_grad = np.zeros((n+1, 1))
    ###

Построим график изменений нормы градиента в зависимости от номера итерации.

In [ ]:
plt.plot(norms)
plt.show()

Графически оценим результаты вычислений.

In [ ]:
print (theta_grad)
yp = h(Xdata1, theta_grad) # Предсказанные значения
plot_data(Xdata, ydata, Xdata, yp)

### Метод Бройдена — Флетчера — Гольдфарба — Шанно (BFGS)

Этот метод реализован в функции `sp.optimize.minimize`. Воспользуйтесь ей, чтобы найти оптимальное значение вектора $\theta$. (Учтите, что команда находит минимум функции от нескольких переменных. Параметр `theta` нужно будет распаковать, а параметры `X` и `y` связать замыканием.)

In [ ]:
# ИСПРАВЬТЕ ИЛИ ДОПОЛНИТЕ КОД НИЖЕ
theta_bfgs = np.zeros((n+1, 1))
###

Графически оценим результаты вычислений.

In [ ]:
print (theta_bfgs)
yp = h(Xdata1, theta_bfgs) # Предсказанные значения
plot_data(Xdata, ydata, Xdata, yp)

### Упрощённая проверка моделей
При помощи каждой из моделей предскажите рост 4-летнего ребёнка.

In [ ]:
# ИСПРАВЬТЕ ИЛИ ДОПОЛНИТЕ КОД НИЖЕ
y4_norm = 0
y4_grad = 0
y4_bfgs = 0
###

## Задача 2. Цены на дом

Требуется найти зависимость стоимости дома (в долларах США) от двух параметров: площади (в кв. футах, первый столбец) и количества спален (второй столбец).

Загрузим исходные данные.

In [ ]:
data = sio.loadmat('prices.mat')
Xdata = data['house']
ydata = data['price']

Дальнейшие вычисления с помощью обоих рассмотренных методов выполните самостоятельно. В этой задаче регрессия множественная, но если вы в предыдущей задачи использовали векторизованные вычисления, то функции не потребуют значительного изменения.

При помощи каждой из моделей предскажите цену на дом площадью 2104 кв. фута с 3 спальнями.

## Выводы

Сделайте выводы о применимости рассмотренных методов для решения задачи линейной регрессии.

- Какие подготовительные действия приходится выполнять перед собственно обучением? Почему?
- Какой из рассмотренных методов проще? Какой точнее?
- Какие преимущества и недостатки у каждого и методов?
- С какими сложностями вы столкнулись в ходе выполнения работы?